In [1]:
import numpy as np
import pandas as pd

Lets write a simple pricing function for a European call option. Assume the stocks follow a lognormal distribution. Then we can compute the price as the expectation
    $$ e^{-rT}\mathbb{E}^\mathbb{P}[C(S, T)] = C(S, 0) $$ 
As a prerequisite, lets figure out the stock price $S_t$ evolution given a Brownian motion $B_t$. The stock price follows the stochastic differential equation
    $$ dS_t = \mu S_t dt + \sigma S_t dB_t $$
We can solve this by letting $X_t=\log S_t$. By Ito's lemma, we get
    $$ dX_t = (\mu-\frac{1}{2}\sigma^2) dt + \sigma dB_t $$
which implies $X_t = X_0 + (\mu-\frac{1}{2}\sigma^2) t + \sigma B_t$. Hence we get that
    $$ S_t = S_0\exp^{(\mu-\frac{1}{2}\sigma^2) t + \sigma B_t} $$
Noting that $B_t$ as a Wiener process has a distribution $B_t \sim N(0,t)=\sqrt{t}N(0,1)$, we can compute this numerically via sampling from the standard normal distribution.

In [4]:
from scipy.integrate import quad
from scipy.stats import norm

def priceEuropeanOption(S0,r,T,sigma,payoff):
    def ST(sample):
        return S0 * np.exp((r-0.5*sigma**2)*T + sigma*np.sqrt(T)*sample)
    R = 10
    def integrand(x):
        return norm.pdf(x) * payoff(ST(x))
    discount = np.exp(-r*T)
    return discount * quad(integrand, -R, R)[0]

In [5]:
# sanity check
def justStock(x):
    return x
priceEuropeanOption(100,0.2,4,0.3,justStock)

100.00000000000001

A european call option has payoff given by a specific strike price $K$.

In [6]:
K = 90
def payoff(x):
    return np.max([x-K, 0])
priceEuropeanOption(100,0.05,2,0.4,payoff)

30.761890184007925